In [ ]:
import pandas as pd
df = pd.read_csv("/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv", encoding='latin-1')
df = df.sample(n=3000, random_state=42)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

In [ ]:
df.head()
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df.head()

In [ ]:
df.shape
df.isnull().sum()

In [ ]:
!pip install googletrans

In [ ]:
!pip install preprocess_kgptalkie

In [ ]:
import preprocess_kgptalkie as ps
df['word_counts'] = df['text'].apply(lambda x: ps.word_count(x))
df['char_count'] = df['text'].apply(lambda x: ps.char_count(x))
df['avg_wordlength'] = df['text'].apply(lambda x: ps.avg_word_len(x))
df['stops_counts'] = df['text'].apply(lambda x: ps.stop_words_count(x))

In [ ]:
df.head()

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words and convert to lowercase (lowercase was already done in a previous step)
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    return ' '.join(tokens)

df['text'] = df['text'].apply(preprocess_text)

In [ ]:
df['text'] = df['text'].str.lower()

In [ ]:
df['target'] = df['target'].replace(4, 1)

In [ ]:
# Get a random sample of 2000 rows from the DataFrame
# Using random_state makes your sample reproducible
# df_sample = df.sample(n=2000, random_state=42)

# Now create your lists from this smaller, sampled DataFrame
X = df['text'].tolist()
y = df['target']

print(f"Selected {len(X)} examples.")

In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len=512):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text= str(self.texts[idx])
    label = torch.tensor(self.labels[idx])

    encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len)

    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'label': label
    }

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'distilbert-base-uncased'
device="cuda"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
model

dataset = CustomDataset(X, y, tokenizer)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to plain lists
X_train = list(X_train)
y_train = list(y_train)
X_test = list(X_test)
y_test = list(y_test)

# Now safe to use in Dataset
train_dataset = CustomDataset(X_train, y_train, tokenizer)
test_dataset = CustomDataset(X_test, y_test, tokenizer)

In [ ]:
from transformers import TrainingArguments, Trainer
batch_size = 8
model_name = "tiwttersentiment"

args = TrainingArguments(
    output_dir="output",
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    eval_strategy='epoch',
    report_to="none",
)

In [ ]:
trainer = Trainer(model = model,
                  args = args,
                  train_dataset=train_dataset,
                  eval_dataset = test_dataset,
                  compute_metrics=compute_metrics,
                  tokenizer = tokenizer
                  )

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(model_name)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Evaluate using Trainer
results = trainer.evaluate(eval_dataset=test_dataset)

# Print standard metrics
print("Test Accuracy:", results['eval_accuracy'])
print("Test F1 Score:", results['eval_f1'])

# 🔄 Make predictions manually to get full outputs
predictions_output = trainer.predict(test_dataset)
preds = np.argmax(predictions_output.predictions, axis=1)
labels = predictions_output.label_ids

# 🧾 Print classification report and confusion matrix
print("\nClassification Report:\n", classification_report(labels, preds))
print("Confusion Matrix:\n", confusion_matrix(labels, preds))

In [ ]:
from transformers import pipeline

text = "I absolutely love this product! It works like a charm."
pipe = pipeline("sentiment-analysis", model=model_name, tokenizer=tokenizer)
pipe(text)

In [ ]:
# Let's create a function to predict sentiment
from transformers import pipeline

# Load our fine-tuned model
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name)

# Define identity terms
male_terms = ['man', 'boy', 'he', 'his']
female_terms = ['woman', 'girl', 'she', 'her']
# You can add more groups, e.g., racial or religious terms
# IMPORTANT: Be mindful and respectful when choosing terms.

# Template sentences
templates = [
    "I am a {} and I am happy.",
    "The {} said they felt good.",
    "This is a story about a {}.",
    "I saw a {} today.",
]

# Test the model
results = []
for template in templates:
    for term_type, terms in [('male', male_terms), ('female', female_terms)]:
        for term in terms:
            sentence = template.format(term)
            prediction = sentiment_pipeline(sentence)[0]
            results.append({
                'group': term_type,
                'term': term,
                'sentence': sentence,
                'prediction': prediction['label'],
                'score': prediction['score']
            })

# Display results in a DataFrame
results_df = pd.DataFrame(results)
print("Bias Test Results:")
print(results_df)

# Analyze the results
# Do sentences with 'male' terms get different predictions than 'female' terms?
print("\nPrediction counts by group:")
print(results_df.groupby('group')['prediction'].value_counts())

In [ ]:
from fairlearn.metrics import MetricFrame, demographic_parity_difference, equalized_odds_difference

# For this example, let's assume the "true" sentiment of our template sentences is POSITIVE.
# This is an assumption to demonstrate the metric.
# We define 'group' by the presence of a gendered term.

# Get predictions for our test sentences (0 for NEGATIVE, 1 for POSITIVE)
y_pred = [1 if r['prediction'] == 'LABEL_1' else 0 for r in results]
# Assume the true label for all these happy sentences is 1 (Positive)
y_true = [1] * len(results_df)
# Define the sensitive feature (male vs. female group)
sensitive_features = results_df['group']

# Now, let's use Fairlearn to calculate metrics
# We use 'selection_rate' which is the percentage of positive predictions (1s)
metrics = {
    'selection_rate': lambda y_true, y_pred: y_pred.mean(),
    'accuracy': lambda y_true, y_pred: (y_true == y_pred).mean()
}

metric_frame = MetricFrame(metrics=metrics,
                           y_true=y_true,
                           y_pred=y_pred,
                           sensitive_features=sensitive_features)

print("\nFairness Metrics (by group):")
print(metric_frame.by_group)

# Calculate the difference in metrics between groups
print("\nMetric Differences (Bias):")
print(metric_frame.difference())